In [555]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp
import random

import os
import sys

## Use this space to write the components of the small cluster simulation algorithm. First write it out in words, then write out the code component

First we have to make up some (smaller scale) data to test things on. This can also serve as the template for making our actual star arrays

In [556]:
random.seed(None)

In [557]:
# intialize array of stars
n = 10

msun = 2e30
G = 6.67430e-11
AU = 1.496e11 
posrange = 40.0 * AU
velrange = 10.0
eta = 2e-4

stars = np.zeros((n, 5), dtype=object)

for i in range(n):
    #build pos and vel arrays
    x = []
    v = []
    for j in range(3):
        x.append(random.uniform(-posrange, posrange))
        v.append(random.uniform(-velrange, velrange))
    stars[i][0] = x #pos
    stars[i][1] = v #vel
    stars[i][2] = [0,0,0,0] #time step, 3 is time of last eval
    stars[i][4] = [random.uniform(0.5*msun, 8*msun)] #mass

In [558]:
# calculate a reasonable epsilon

# get average of all stars masses
avg = 0
for i in range(n):
    avg = avg + stars[i][4][0]
avg = avg / n

# calc a (although we will just choose it to be 1 AU)
m = 2 * avg
TE = 6.822e+40
a = (-G * m**2)/(2 * TE)

# set epsilon 
epsilon = AU - 5e10

In [559]:
# calculate initial force on all stars

def dot(vec1, vec2):
    '''perform dot procuct'''
    return (vec1[0]*vec2[0]) + (vec1[1]*vec2[1]) + (vec1[2]*vec2[2])

def starposvec(stars,i,j):
    '''returns position vector from star i to star j'''
    return np.subtract(stars[i][0][:],stars[j][0][:])

def starvelvec(stars,i,j):
    '''returns velocity vector from star i to star j'''
    return np.subtract(stars[i][1][:],stars[j][1][:])

def stardist(i,j):
    '''calculates magnitude of distance between i and j'''
    dists = starposvec(stars,i,j)
    return np.sqrt(dists[0]**2 + dists[1]**2 + dists[2]**2)

def f_ij(stars,i,j):
    '''returns the force vector between just i and j'''
    pos = starposvec(stars,i,j)
    dist = stardist(i,j)
    return -stars[j][4][0]*pos/((dist**2 + epsilon**2)**(3/2))

def fd_ij(stars,i,j):
    '''returns force derivative vector between just i and j'''
    velvec = starvelvec(stars,i,j)
    posvec = starposvec(stars,i,j)
    dist = stardist(i,j)
    return -stars[j][4][0]*((velvec/(dist**2 + epsilon**2)**(3/2))-((3*posvec*dot(posvec,velvec))/((dist**2 + epsilon**2)**(5/2))))


def init_force(stars):
    '''calculate initial force on particles'''
    force = np.zeros((n,3))
    for i in range(n):
        for j in range(n):
            if i!=j:
                vec = starposvec(stars,i,j)
                dist = stardist(i,j)
                force[:][i] = force[:][i] + (-G * stars[j][4][0] * vec)/(np.sqrt((dist**2 + epsilon**2)**3))
    return force

def init_force_mag(force):
    '''calculate inital force magnitude on each particle'''
    force_mag = np.zeros(n)
    for i in range(n):
        force_mag[i] = np.sqrt(force[i][0]**2 + force[i][1]**2 + force[i][2]**2)
    return force_mag

def init_force_fd(stars):
    '''calculate initial force first derivative'''
    force_fd = np.zeros((n,3))
    for i in range(n):
        for j in range(n):
            if i!=j:
                posvec = starposvec(stars,i,j)
                velvec = starvelvec(stars,i,j)
                dist = stardist(i,j)
                force_fd[:][i] = force_fd[:][i] + (-stars[j][4][0])*((velvec/((dist**2 + epsilon**2)**(3/2)))-((3*posvec*dot(posvec,velvec))/((dist**2 + epsilon**2)**(5/2))))
    return force_fd

def init_force_sd(stars):
    '''calculate initial force second derivative'''
    force_sd = np.zeros((n,3))
    for i in range(n):
        for j in range(n):
            if i!=j:
                posvec = starposvec(stars,i,j)
                velvec = starvelvec(stars,i,j)
                dist = stardist(i,j)

                t1 = (f_ij(stars,i,j)/((dist**2 + epsilon**2)**(3/2)))
                t2 = ((6*velvec*dot(posvec,velvec))/(dist**5))
                t3 = (((3*posvec)/((dist**2 + epsilon**2)**(5/2)))*(((5*(dot(posvec,velvec))**2)/(dist**2 + epsilon**2))-(dot(velvec,velvec))-(dot(posvec,f_ij(stars,i,j)))))

                force_sd[:][i] = force_sd[:][i] + (-stars[j][4][0])*(t1-t2+t3)
    return force_sd

def init_force_td(stars):
    '''calculate initial force third derivative'''
    force_td = np.zeros((n,3))
    for i in range(n):
        for j in range(n):
            if i!=j:
                posvec = starposvec(stars,i,j)
                velvec = starvelvec(stars,i,j)
                dist = stardist(i,j)
                rsd = f_ij(stars,i,j)
                rtd = fd_ij(stars,i,j)

                t1 = rtd/((dist**2 + epsilon**2)**(3/2))
                t2 = (9*rsd*dot(posvec,velvec))/((dist**2 + epsilon**2)**(5/2))
                t3 = (velvec/((dist**2 + epsilon**2)**(5/2)))*((9*dot(velvec,velvec))+(9*dot(posvec,rsd))-((45*(dot(posvec,velvec))**2)/(dist**2 + epsilon**2)))
                t4 = (posvec/((dist**2 + epsilon**2)**(5/2)))*((3*dot(posvec,rtd))+(9*dot(velvec,rsd))-((45*dot(posvec,velvec)*dot(posvec,rsd))/(dist**2 + epsilon**2))-((45*dot(posvec,velvec)*dot(velvec,velvec))/(dist**2 + epsilon**2))+((105*dot(posvec,velvec)**3)/((dist**2 + epsilon**2)**2)))

                force_td[:][i] = force_td[:][i] + (-stars[j][4][0])*(t1-t2-t3-t4)
    return force_td

In [560]:
# initial force and first, second, third derivs
f0 = init_force(stars)
f0_fd = init_force_fd(stars)
f0_sd = init_force_sd(stars)
f0_td = init_force_td(stars)

In [561]:
def init_dt(stars):
    '''initialize beginning dt as laid out in Aarseth eq 7'''
    for i in range(n):
        f0_mag = init_force_mag(f0)
        f03_mag = init_force_mag(f0_td)
        dt0 = (eta*((6*f0_mag[i])/(f03_mag[i])))**(1/3)
        stars[i][2] = [dt0, dt0, dt0, dt0]

In [562]:
init_dt(stars)

1) Find the particle (star) to be given the next force evalutaion
2) Advance time to new time step

In [563]:
t = 0
def find_min_dt(stars):
    dt = stars[0][2][3]
    ti = stars[0][3]
    min = dt + ti
    part = 0
    for i in range(n):
        dt = stars[i][2][3]
        ti = stars[i][3]
        if dt+ti < min:
            part = i
            min = dt+ti
    return dt, part

dt_alpha, alpha = find_min_dt(stars)
t_old = t
t = t_old + dt_alpha

3) Change the position of all of the particles except the chosen particle to the new time t_new, using only the first two terms of the force polynomial. If you integrate this force formula twice with respect to time, you get the change in position

Note: must keep track of last time of force eval for each particle!!! different for each particle!!

Ok I'm realizing that to do this, you have to have an idea of the force on each particle already...let's write a calculating initial 
force function

I'm realizing that to do this we also need an automated way of calculating B, C, D, E. See Aarseth (1971) for this

In [564]:
def force_fd(stars,alpha):
    '''calculate force first derivative for a single particle'''
    force_fd = np.zeros(3)
    for j in range(n):
        if j!=alpha:
            posvec = starposvec(stars,alpha,j)
            velvec = starvelvec(stars,alpha,j)
            dist = stardist(alpha,j)
            force_fd = force_fd + (-stars[j][4][0])*((velvec/((dist**2 + epsilon**2)**(3/2)))-((3*posvec*dot(posvec,velvec))/((dist**2 + epsilon**2)**(5/2))))
    return force_fd

def force_sd(stars, alpha):
    '''calculate force second derivative for a single particle'''
    force_sd = np.zeros(3)
    for j in range(n):
        if j!=alpha:
            posvec = starposvec(stars,alpha,j)
            velvec = starvelvec(stars,alpha,j)
            dist = stardist(alpha,j)

            t1 = (f_ij(stars,alpha,j)/((dist**2 + epsilon**2)**(3/2)))
            t2 = ((6*velvec*dot(posvec,velvec))/((dist**2 + epsilon**2)**(5/2)))
            t3 = (((3*posvec)/((dist**2 + epsilon**2)**(5/2)))*(((5*(dot(posvec,velvec))**2)/(dist**2 + epsilon**2))-(dot(velvec,velvec))-(dot(posvec,f_ij(stars,alpha,j)))))

            force_sd = force_sd + (-stars[j][4][0])*(t1-t2+t3)
    return force_sd

def force_td(stars, alpha):
    '''calculate force third derivative for a single particle'''
    force_td = np.zeros(3)
    for j in range(n):
        if j!=alpha:
            posvec = starposvec(stars,alpha,j)
            velvec = starvelvec(stars,alpha,j)
            dist = stardist(alpha,j)
            rsd = f_ij(stars,alpha,j)
            rtd = fd_ij(stars,alpha,j)

            t1 = rtd/((dist**2 + epsilon**2)**(3/2))
            t2 = (9*rsd*dot(posvec,velvec))/((dist**2 + epsilon**2)**(5/2))
            t3 = (velvec/((dist**2 + epsilon**2)**(5/2)))*((9*dot(velvec,velvec))+(9*dot(posvec,rsd))-((45*(dot(posvec,velvec))**2)/(dist**2 + epsilon**2)))
            t4 = (posvec/((dist**2 + epsilon**2)**(5/2)))*((3*dot(posvec,rtd))+(9*dot(velvec,rsd))-((45*dot(posvec,velvec)*dot(posvec,rsd))/(dist**2 + epsilon**2))-((45*dot(posvec,velvec)*dot(velvec,velvec))/(dist**2 + epsilon**2))+((105*dot(posvec,velvec)**3)/((dist**2 + epsilon**2)**2)))

            force_td = force_td + (-stars[j][4][0])*(t1-t2-t3+t4)
    return force_td

In [565]:
def fhat1_calc(stars,i):
    '''calculate fhat1 for particle i'''
    f0_fd = force_fd(stars,i)
    f0_sd = force_sd(stars,i)
    f0_td = force_td(stars,i)
    dt3 = stars[i][2][2]
    return f0_fd - (0.5*f0_sd*dt3) + ((1/6)*f0_td*(dt3**2))

def fhat2_calc(stars,i):
    '''calculate fhat2 for particle i'''
    f0_sd = force_sd(stars,i)
    f0_td = force_td(stars,i)
    dt2 = stars[i][2][1]
    dt3 = stars[i][2][2]
    return (0.5*f0_sd*((dt2+dt3)/(dt3))) - ((1/6)*f0_td*(dt2+(2*dt3))*(dt2+dt3)/(dt3))

def fhat3_calc(stars,i):
    '''calculate fhat3 for particle i'''
    fhat2 = fhat2_calc(stars,i)
    f0_td = force_td(stars,i)
    dt1 = stars[i][2][0]
    dt2 = stars[i][2][1]
    dt3 = stars[i][2][2]
    return (fhat2*((dt2**2)-(dt1*dt3))/((dt2+dt3)*dt2*dt3)) + ((1/6)*f0_td*(dt1+dt2+dt3)*(dt1+dt2)/(dt2*dt3))

def B_calc(stars, i):
    '''calculate the prefactor of the linear term in the polynomial, eq4 in Aarseth'''
    return fhat1_calc(stars, i)

def C_calc(stars, i):
    '''calculate the prefactor of the quadratic term in the polynomial, eq4 in Aarseth'''
    dt3 = stars[i][2][2]
    dt2 = stars[i][2][1]
    return dt3*(fhat2_calc(stars, i))/(dt2+dt3)

def D_calc(stars, i):
    '''calculate the prefactor of the cubic term in the polynomial, eq4 in Aarseth'''
    dt1 = stars[i][2][0]
    dt2 = stars[i][2][1]
    dt3 = stars[i][2][2]
    fhat2 = fhat2_calc(stars, i)
    fhat3 = fhat3_calc(stars, i)
    return (dt2*dt3*fhat3/((dt1+dt2+dt3)*(dt1+dt2))) - (((dt2**2)-(dt1*dt3))*fhat2/((dt1+dt2+dt3)*(dt1+dt2)*(dt2+dt3)))

In [566]:
def dr(stars, i, tr):
    '''calculate change in positon of particle i != alpha'''
    vel = stars[i][1]
    f0 = init_force(stars)[i]
    B = B_calc(stars, i)
    return ([x*tr for x in vel]) + (0.5*f0*(tr**2)) + ((1/6)*B*(tr**3))

def update_positions(stars, alpha, t):
    for i in range(n):
        ti = stars[i][3]
        if i != alpha:
            tr = t - ti
            stars[i][0] = stars[i][0] + dr(stars, i, tr)

In [567]:
update_positions(stars, alpha, t)

4) Predict the new position of the particle alpha by two time integrations of the cubic expression for the force on it

In [568]:
def dr0(stars, alpha, tr):
    '''calculate change in position of alpha based on cubic expression for force'''
    vel = stars[alpha][1]
    F0 = f0[alpha]
    B = B_calc(stars, alpha)
    C = C_calc(stars, alpha)
    D = D_calc(stars, alpha)
    return ([x*tr for x in vel]) + (0.5*F0*(tr**2)) + ((1/6)*B*(tr**3)) + ((1/12)*C*(tr**4)) + ((1/20)*D*(tr**5))

def update_alpha_pos(stars, alpha, t):
    '''update the position of the particle alpha based on the cubic expression for force'''
    ti = stars[alpha][3]
    tr = t - ti
    stars[alpha][0] = stars[alpha][0] + dr0(stars, alpha, tr)

In [569]:
update_alpha_pos(stars, alpha, t)

5) Evaluate the force per unit mass on alpha by summing contributions from all other particles, using their new positions
6) Calculate E and correct the position of alpha by adding the movement due to E

In order to calculate E, we need a way to calculate f_alpha_fd, f_alpha_sd, f_alpha_td, and fhat2 for just one particle--write those functions here

In [570]:
def f_alpha(stars, alpha):
    '''calculate the force on just alpha using the updated positions of all of the other particles'''
    force = np.zeros(3)
    for j in range(n):
        if j!=alpha:
            vec = starposvec(stars,alpha,j)
            dist = stardist(alpha,j)
            force = force + (-G * stars[j][4][0] * vec)/(np.sqrt((dist**2 + epsilon**2)**3))
    return force

def f4hat2_calc(stars,alpha):
    '''calculate fhat2 for particle alpha, with the updated force'''
    f0_sd = force_sd(stars, alpha)
    f0_td = force_td(stars, alpha)
    dt2 = stars[alpha][2][1]
    dt3 = stars[alpha][2][2]
    return (0.5*f0_sd*((dt2+dt3)/(dt3))) - ((1/6)*f0_td*(dt2+(2*dt3))*(dt2+dt3)/(dt3))

def E_calc(stars, i):
    '''calculate the prefactor of the quartic term in the polynomial, eq4 in Aarseth'''
    dt1 = stars[i][2][0]
    dt2 = stars[i][2][1]
    dt3 = stars[i][2][2]
    dt4 = stars[i][2][3]
    f4hat2 = f4hat2_calc(stars, i)
    fhat2 = fhat2_calc(stars, i)
    D = D_calc(stars, i)
    return (((dt4*f4hat2/(dt3+dt4))-(dt3*fhat2/(dt2+dt3)))/((dt1+dt2+dt3+dt4)*(dt2+dt3+dt4))) - (D/(dt1+dt2+dt3+dt4))

def update_alpha_pos_E(stars, alpha, t):
    '''update the position of the particle alpha by adding E term'''  
    E = E_calc(stars, alpha)
    ti = stars[alpha][3]
    tr = t - ti
    add = (1/30)*E*(tr**6)
    stars[alpha][0] = stars[alpha][0] + add

In [571]:
Fa = f_alpha(stars, alpha)
update_alpha_pos_E(stars, alpha, t)

7) Calculate the new velocity on particle alpha by one time integration of the quartic expression for the force. 

In [572]:
def dv0(stars, alpha, tr):
    '''calculate change in velocity of alpha based on quartic expression for force'''
    F0 = f_alpha(stars, alpha)
    B = B_calc(stars, alpha)
    C = C_calc(stars, alpha)
    D = D_calc(stars, alpha)
    E = E_calc(stars, alpha)
    return F0*tr + (0.5*B*(tr**2)) + ((1/3)*C*(tr**3)) + ((1/4)*D*(tr**4)) + ((1/5)*D*(tr**5))

def update_alpha_vel(stars, alpha, t):
    '''update the velocity of the particle alpha based on the quartic expression for force'''
    ti = stars[alpha][3]
    tr = t - ti
    stars[alpha][1] = stars[alpha][1] + dv0(stars, alpha, tr)

In [573]:
update_alpha_vel(stars, alpha, t)

In [574]:
stars[alpha][2]

[10.780936674991056,
 10.780936674991056,
 10.780936674991056,
 10.780936674991056]

8) Calculate the time to the next force evaluation for particle alpha

In [575]:
def mag(force):
    '''calculate the magnitude of some 3 element vector'''
    return np.sqrt(force[0]**2 + force[1]**2 + force[2]**2)

def calc_new_timestep(stars, alpha):
    '''calculate the new timestep size for alpha, and reassign each of the old timesteps'''
    F0 = mag(f_alpha(stars, alpha))
    B = mag(B_calc(stars, alpha))
    D = mag(D_calc(stars, alpha))
    E = mag(E_calc(stars, alpha))
    dtnew = (eta*(F0 + B*dt_alpha)/(D + E*dt_alpha))**(1/3)
    
    stars[alpha][2][0] = stars[alpha][2][1]
    stars[alpha][2][1] = stars[alpha][2][2]
    stars[alpha][2][2] = stars[alpha][2][3]
    stars[alpha][2][3] = dtnew
    stars[alpha][3] = t


In [576]:
calc_new_timestep(stars, alpha)

In [577]:
stars[alpha]

array([array([2.03594526e+12, 1.92613732e+12, 4.90274430e+12]),
       array([149.30528662, -47.26185184, -11.82554806]),
       list([10.780936674991056, 10.780936674991056, 10.780936674991056, 339.7771563262896]),
       14.6438702101977, list([5.563285575569554e+30])], dtype=object)